In [1]:
!nvidia-smi

Mon May 16 10:50:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 510.60.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     On   | 00000000:00:06.0 Off |                  Off |
| N/A   31C    P0    49W / 250W |      3MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 6000     On   | 00000000:00:07.0 Off |                  Off |
| N/A   

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained("huggingface/CodeBERTa-small-v1")
model = AutoModel.from_pretrained("huggingface/CodeBERTa-small-v1")

/home/hahajjjun/anaconda3/envs/molly/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at huggingface/CodeBERTa-small-v1 were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassifi

In [3]:
import random
import numpy as np
import os
import pandas as pd
import torch.nn as nn
import warnings 
warnings.filterwarnings("ignore")
from tqdm import tqdm
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
seed_everything(42)

In [4]:
def load_data(path):
    #TRAIN = os.path.join(path, 'train.csv')
    TRAIN = os.path.join(path, 'train_data.csv')
    VALID = os.path.join(path, 'val_data.csv')
    TEST = os.path.join(path, 'test.csv')
    SS = os.path.join(path, 'sample_submission.csv')
    train = pd.read_csv(TRAIN)
    valid = pd.read_csv(VALID)
    test = pd.read_csv(TEST)
    sample_submission = pd.read_csv(SS)
    return train,valid,test,sample_submission
    #return train, test, sample_submission

In [5]:
train, valid, test, ss = load_data('/home/hahajjjun/Junha Park/DACON_PYNLI')

In [6]:
def text_clean(df):
    df["text_sum"] = "[CLS] "+df["code1"]+" [SEP] "+df["code2"]+" [SEP]"
    df = df[['text_sum','similar']]
    return df

In [7]:
train = text_clean(train)
valid = text_clean(valid)

In [8]:
def codebert_transform(text):
    transform = tokenizer(text,pad_to_max_length=True,truncation=True,max_length=512, return_tensors='pt',add_special_tokens=False)
    return transform

class customDataset(Dataset):
    def __init__(self,dataset,mode='train',transform=codebert_transform):
        super(customDataset, self).__init__()
        self.mode = mode
        self.dataset = dataset
        self.transform = transform
    
    def __getitem__(self, idx):
        text = self.dataset['text_sum'].iloc[idx]
        tokens = self.transform(text)
        token_ids = tokens['input_ids'][0]
        attn_masks = tokens['attention_mask'][0]
        #token_type_ids = tokens['token_type_ids'][0]

        if self.mode == 'test':
          return token_ids,attn_masks #,token_type_ids
        else: 
          labels = self.dataset['similar'].iloc[idx]
          return token_ids,attn_masks,labels #token_type_ids, labels

    def __len__(self):
        return(len(self.dataset))

In [9]:
    class codeBERTclassifier(nn.Module):
        def __init__(self, model, hidden_size = 768, num_classes=2, params=None,  freeze=False):
            super(codeBERTclassifier, self).__init__()
            self.model = model
            self.freeze = freeze

            if self.freeze:
                for p in self.model.parameters():
                    p.requires_grad = False


            self.classifier = nn.Linear(hidden_size , 256)
            self.dropout = nn.Dropout(p=0.5)
            self.fc_layer1 = nn.Linear(256,128)
            self.fc_layer2 = nn.Linear(128,num_classes)
            self.softmax = nn.Softmax()


        def forward(self, input_ids, attn_masks):

            _,pooler = self.model(input_ids, attn_masks, return_dict=False)
            output1 = self.classifier(pooler)
            output2 = self.fc_layer1(output1)
            output3 = self.fc_layer2(self.dropout(output2))
            return output3

In [10]:
device = torch.device("cuda:0")
lr = 1e-5
batch_size= 64
warmup_ratio = 0.06
num_epochs = 1
log_interval = 10000

In [11]:
model = codeBERTclassifier(model).to(device)
#model=nn.DataParallel(model).to(device)
optimizer = AdamW(model.parameters(), lr=lr)

#random_idx = random.sample(range(len(train)), len(train))
#train_idx = random_idx[:int(len(train)*0.9)]
#val_idx = random_idx[int(len(train)*0.9):]
#train_split = train.iloc[train_idx]
#val_split = train.iloc[val_idx]
best_models = []

#train_dataset = customDataset(train_split,'train')
#valid_dataset = customDataset(val_split,'train')
#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
train_dataset = customDataset(train,'train')
valid_dataset = customDataset(valid,'train')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

total_steps = len(train_loader) * num_epochs
warmup_step = int(total_steps * warmup_ratio)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
valid_loss_min = 0.4
valid_acc_max = 0.8
    
for epoch in range(num_epochs):
    batches = 0
    total_loss = 0.0
    correct = 0
    total =0
    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader): # token_type_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = y_batch.to(device)
        #y_pred = torch.transpose(model(input_ids_batch.to(device), attention_masks_batch.to(device), token_type_batch.to(device)), 0, 1)[0]
        y_pred = model(input_ids_batch.to(device), attention_masks_batch.to(device))
        loss = F.cross_entropy(y_pred, y_batch)
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        total_loss += loss.item()
        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum()
        total += len(y_batch)
        batches += 1
        if batches % log_interval == 0:
            print("Batch Loss: ", total_loss / batches, "Accuracy: ", correct.float() / total)
    scheduler.step()
    val_loss = []
    val_acc = []
    
    # VALIDATION #
    for input_ids_batch, attention_masks_batch, y_batch in tqdm(valid_loader):

        model.eval()
        with torch.no_grad():
            y_batch = y_batch.to(device)
            y_pred = model(input_ids_batch.to(device), attention_masks_batch.to(device))
            valid_loss = F.cross_entropy(y_pred, y_batch)
            valid_loss = valid_loss.cpu().detach().numpy()
            preds = torch.argmax(y_pred,1)
            preds = preds.cpu().detach().numpy()
            y_batch = y_batch.cpu().detach().numpy()
            batch_acc = (preds==y_batch).mean()
            val_loss.append(valid_loss)
            val_acc.append(batch_acc)


    val_loss = np.mean(val_loss)
    val_acc = np.mean(val_acc)

    print(f'Epoch: {epoch} - valid Loss: {val_loss:.6f} - valid_acc : {val_acc:.6f}')
    if valid_acc_max < val_acc:
        valid_acc_max = val_acc
        best_models.append(model)
        torch.save(model.state_dict(), f'Junha Park/DACON_PYNLI/weights/codeBERT-{len(best_models)}.pth') 
        print('model saved, model val acc : ',val_acc)
        print('best models size : ',len(best_models))

  0%|          | 63/160431 [00:36<25:33:49,  1.74it/s]


KeyboardInterrupt: 

In [12]:
def text_clean_test(df):
    df["text_sum"] = "[CLS] "+df["code1"]+" [SEP] "+df["code2"]+" [SEP]"
    df = df[['text_sum', 'code1']]
    return df
test = text_clean_test(test)
test_dataset = customDataset(test, 'test')
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
bestm = model
bestm.eval()
answer = []
with torch.no_grad():
    for input_ids_batch, attention_masks_batch in tqdm(test_dataloader):
        y_pred =bestm(input_ids_batch.to(device), attention_masks_batch.to(device)).detach().cpu().numpy()
        answer.extend(y_pred)

100%|██████████| 179700/179700 [41:30<00:00, 72.16it/s] 


In [14]:
p = []
for item in answer:
    p.append(item.argmax())

In [15]:
ss['similar'] = p

In [16]:
ss.to_csv('submission_codeBERTa.csv', index = False)